In [1]:
import sys
import os

os.environ["APPLIED_AI_CONF"] = "../../conf/app.toml"

if os.path.abspath("../..") not in sys.path:
    sys.path.insert(0, os.path.abspath("../.."))

from google.cloud.ml.applied.attributes import attributes
from google.cloud.ml.applied.categories import category
from google.cloud.ml.applied.model import domain_model as m
from google.cloud.ml.applied.images import image_to_text
from google.cloud.ml.applied.marketing import marketing

## Generate Product Categories

Based on any product description and (optional) image, we perform the following:

* Create multimodal embeddings (text and image) from the input description and image
* Perform nearest neighbor search on existing product catalog embeddings in Vertex Vector Search to fetch x nearest product ids
* Lookup existing product categories from the retrieved ids.
* Use an LLM to rank returned categories

In [2]:
category.retrieve_and_rank(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)"
)

CategoryList(values=[['Watches', 'Wrist Watches', 'Timewel Wrist Watches']])

In [5]:
category.retrieve_and_rank(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    image = "gs://genai-product-catalog/flipkart_20k_oct26/dbdac18a8ee5a8a48238b9685c96e90a_0.jpg"
)

CategoryList(values=[['Watches', 'Wrist Watches', 'Timewel Wrist Watches'], ['Watches', 'Wrist Watches', 'Times Wrist Watches']])

## Generate Product Attributes

We utilize an LLM to generate product attributes to further enhance a product listing making it more effective for search and recommendations usecases.

Using the same mechanism above, we can use a product description and an image to generate these attributes

In [6]:
attributes.retrieve_and_generate_attributes(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
)

{'Brand': 'Timewel',
 'Model': '1100-N1949_S',
 'Gender': 'Women',
 'Price': 'Rs.855',
 'Discount': 'Great',
 'Product Type': 'Analog Watch',
 'Shipping': 'Free',
 'COD': 'Yes',
 'Guarantee': '30 Day Replacement'}

In [7]:
product_attiributes = attributes.retrieve_and_generate_attributes(
    desc = "Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    image = "gs://genai-product-catalog/flipkart_20k_oct26/dbdac18a8ee5a8a48238b9685c96e90a_0.jpg"
)
product_attiributes

{'Brand': 'Timewel',
 'Model': '1100-N1949_S',
 'Gender': 'Women',
 'Price': 'Rs.855',
 'Discount': 'Great',
 'Product Type': 'Analog Watch',
 'Shipping': 'Free',
 'Cash on Delivery': 'Yes',
 'Replacement Guarantee': '30 Days'}

## Generate Product Image Attributes
We can also use an LVM to analyze a product image an extract product attributes

In [8]:
test_image = m.ImageRequest(image="gs://genai-product-catalog/flipkart_20k_oct26/dbdac18a8ee5a8a48238b9685c96e90a_0.jpg")
image_to_text.image_to_attributes(test_image)

InvalidArgument: 400 Request contains an invalid argument.

## Generate Marketing Description
We use the product description as well as the above generated product attributes to generate a product marketing description

In [10]:
marketing_request = m.MarketingRequest(
    description="Timewel 1100-N1949_S Analog Watch - For Women - Buy Timewel 1100-N1949_S Analog Watch - For Women 1100-N1949_S Online at Rs.855 in India Only at Flipkart.com. - Great Discounts, Only Genuine Products, 30 Day Replacement Guarantee, Free Shipping. Cash On Delivery!)",
    attributes=product_attiributes
)

marketing_copy = marketing.generate_marketing_copy(marketing_request)
print(marketing_copy.text)

ValidationError: 1 validation error for MarketingRequest
attributes
  value is not a valid list (type=type_error.list)